# Sistem Penjadwalan Praktikum Otomatis menggunakan Algoritma Genetika dan Tabu Search.

# Setup

In [24]:
import os
import sys
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "LabTimetablingAPI.settings")
django.setup()
from django.conf import settings
#set debug to true
settings.DEBUG = True

#logging
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


## Data

Import data dari model yang telah dibuat sebelumnya.

```python

In [25]:
from scheduling_data.models import Laboratory, Module, Participant, Group, Assistant, Chapter

Ekstrak data dari model yang telah dibuat sebelumnya.

```python

In [26]:
class LaboratoryData:

    def __init__(self, lab = Laboratory):
        self._lab = lab
        self._laboratories = lab.objects.all()

    @property
    def laboratories(self): 
        return self._laboratories
    
    def get_assistant(self,id):
        laboratory = self._laboratories.filter(id=id).first()
        assistants = laboratory.assistants.all()
        return assistants
    
    def get_module(self,id):
        laboratory = self._laboratories.filter(id=id).first()
        modules = laboratory.modules.all()
        return modules
    
class ModuleData:

    def __init__(self, module = Module):
        self._module = module
        self._modules = module.objects.all()

    @property
    def modules(self): 
        return self._modules
    
    def get_module(self,id):
        module = self._modules.filter(id=id).first()
        return module
    
    def get_module_chapter(self, id):
        module = self.get_module(id)
        return module.chapters.all()
    
    def get_group(self, id):
        module = self.get_module(id)
        return module.groups.all()
    
    def get_laboratory(self, id):
        module = self.get_module(id)
        return module.laboratory
    
        
class ChapterData:

    def __init__(self, chapter = Chapter):
        self._chapter = chapter
        self._chapters = chapter.objects.all()

    @property
    def chapters(self): 
        return self._chapters
    
    def get_chapter(self,id):
        chapter = self._chapters.filter(id=id).first()
        return chapter
    
    def get_chapter_module(self, id):
        chapter = self.get_chapter(id)
        return chapter.module

class ParticipantData:

    def __init__(self, participant = Participant):
        self._participant = participant
        self._participants = participant.objects.all()
    
    @property
    def participants(self):
        return self._participants
    
    def group(self, id):
        group_memberships = self._participants.filter(id=id).first().group_memberships.all()
        groups = []
        for group_membership in group_memberships:
            group = group_membership.group
            groups.append(group)
        return groups
    
class GroupData:
    def __init__(self, group = Group):
        self._group = group
        self._groups = group.objects.all()

    @property
    def groups(self):
        return self._groups
    
    def get_group(self, id):
        group = self._groups.filter(id=id).first()
        return group
    
    def get_module_group(self, module_id):
        groups = self._groups.filter(module_id=module_id)
        return groups
    
    def participants(self, id):
        group = self._groups.filter(id=id).first()
        group_memberships = group.group_memberships.all()
        participants = []
        for group_membership in group_memberships:
            participant = group_membership.participant
            participants.append(participant)
        return participants
    
    def get_participant_schedule(self, id):
        participants = self.participants(id)
        participant_schedule = []
        for participant in participants:
            participant_schedule.append(participant.regular_schedule)
        return participant_schedule
    
    def get_group_schedule(self, id):
        participant_schedule = self.get_participant_schedule(id)
        if len(participant_schedule) == 0:
            return {}
        days = participant_schedule[0].keys()
        merged_schedule = {day: {} for day in days}
        for day in days:
            for time_slot in participant_schedule[0][day]:
                is_available = all(schedule[day][time_slot] for schedule in participant_schedule)
                merged_schedule[day][time_slot] = is_available
        return merged_schedule
                
            
class AssistantData:

    def __init__(self, assistant = Assistant):
        self._assistant = assistant
        self._assistants = assistant.objects.all()

    @property
    def assistants(self):
        return self._assistants

    
    def get_assistant(self, id):
        assistant = self._assistants.filter(id=id).first()
        return assistant
    
    def get_assistant_modules(self,id):
        assistant = self._assistant.filter(id=id).first()
        assistant_memberships = assistant.assistant_memberships.all()
        modules = []
        for assistant_membership in assistant_memberships:
            module = assistant_membership.module
            modules.append(module)
        return modules

In [27]:
group_data = GroupData().groups
group_id = [group.id for group in group_data]
len(group_id)


36

In [28]:
# import timeit
# import cProfile
# import pstats

# group_data = GroupData()

# group_id = [group.id for group in group_data.groups]

# def get_group_schedule(group_id = group_id, group_data = group_data):
#     for id in group_id:
#         group_data.get_group_schedule(id)
#     return group_id

# pr = cProfile.Profile()
# pr.enable()
# start = timeit.default_timer()
# #do it 10000000 times
# for i in range(1000):
#     get_group_schedule()
# stop = timeit.default_timer()
# pr.disable()

Constants

```python

In [29]:
# Constant.py
class Constant:
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    shifts = ["Shift1", "Shift2", "Shift3", "Shift4", "Shift5", "Shift6"]

# Algoritma Genetika

## Gene Representation

```python

In [30]:
from collections import namedtuple
TimeSlot = namedtuple("TimeSlot", ["date", "day", "shift"])

class Gene:
    def __init__(self, laboratory, module, module_chapter, group, assistant, time_slot: TimeSlot):
        self.laboratory = laboratory
        self.module = module
        self.module_chapter = module_chapter
        self.group = group
        self.assistant = assistant
        self.time_slot = time_slot
        
    def __repr__(self):
        return f"Gene(lab={self.laboratory}, module={self.module}, module_chapter={self.module_chapter}, group={self.group}, assistant={self.assistant}, time_slot={self.time_slot})"
    
    def __eq__(self, other):
        return self.laboratory == other.laboratory and self.module == other.module and self.module_chapter == other.module_chapter and self.group == other.group and self.assistant == other.assistant and self.time_slot == other.time_slot

Test Gene Representation

```python

In [31]:
# Test Gene

# Assigning data to variables
lab_data = LaboratoryData()
module_data = ModuleData()
participant_data = ParticipantData()
group_data = GroupData()
assistant_data = AssistantData()
chapter_data = ChapterData()

gene = Gene(lab_data.laboratories[0].id, module_data.modules[0].id, chapter_data.chapters[0].id, group_data.groups[0].id,assistant_data.assistants[0].id, TimeSlot("2021-01-01", "Friday", "Shift1"))
gene2 = Gene(lab_data.laboratories[0].id, module_data.modules[0].id, chapter_data.chapters[0].id, group_data.groups[1].id, assistant_data.assistants[0].id, TimeSlot("2021-01-01", "Friday", "Shift2"))
gene3 = Gene(lab_data.laboratories[0].id, module_data.modules[0].id, chapter_data.chapters[0].id, group_data.groups[2].id, assistant_data.assistants[0].id, TimeSlot("2021-01-01", "Monday", "Shift1"))

### Gene Level Constraints

```python

In [32]:
# Constraint Class for Constraint Function
# 1. Ensure that each chapter is assigned to the correct module.
# 2. Ensure that each module is assigned to the correct lab.
# 3. Ensure that each group is assigned to the correct module.
# 4. Ensure that each assistant is assigned to the correct lab.

# import LaboratoryData, ModuleData, GroupData, AssistantData, ChapterData

class ConstraintChecker:
    def __init__(self, gene : Gene, **kwargs):
        kwargs.setdefault("lab_data", LaboratoryData())
        kwargs.setdefault("module_data", ModuleData())
        kwargs.setdefault("group_data", GroupData())
        kwargs.setdefault("assistant_data", AssistantData())
        kwargs.setdefault("chapter_data", ChapterData())
        self.gene = gene
        self.lab_data = lab_data
        self.module_data = module_data
        self.group_data = group_data
        self.assistant_data = assistant_data
        self.chapter_data = chapter_data

    def chapter_module_constraint(self):
        module = self.module_data.get_module(self.gene.module)
        chapter = self.chapter_data.get_chapter(self.gene.module_chapter)
        return module == chapter.module
    
    def module_lab_constraint(self):
        module = self.module_data.get_module(self.gene.module)
        return module.laboratory.id == self.gene.laboratory
    
    def group_module_constraint(self):
        module = self.module_data.get_module(self.gene.module)
        group = self.group_data.get_group(self.gene.group)
        return module == group.module
    
    def assistant_lab_constraint(self):
        assistant = self.assistant_data.get_assistant(self.gene.assistant)
        return assistant.laboratory.id == self.gene.laboratory
    
    def schedule_constraint(self):
        return group_data.get_group_schedule(self.gene.group)[self.gene.time_slot.day][self.gene.time_slot.shift]
    
    def check(self):
        return self.chapter_module_constraint() and self.module_lab_constraint() and self.group_module_constraint() and self.assistant_lab_constraint()

Test Gene Level Constraints

```python

In [33]:
    # Test ConstraintChecker
ConstraintChecker(gene).check()

True

## Chromosome Representation

```python

In [34]:
# Chromosome.py
# Chromosome Representation
# 1. Gene
# 2. Fitness

class Chromosome:
    def __init__(self, genes: list):
        self.genes = genes
        self.fitness = None

    def __repr__(self):
        return f"Chromosome(genes={self.genes}, fitness={self.fitness})"
    
    def __str__(self):
        return f"Chromosome(genes={self.genes}, fitness={self.fitness})"
    
    def __len__(self):
        return len(self.genes)
    
    def __eq__(self, other):
        #check for each gene if it is equal
        return all(gene == other_gene for gene, other_gene in zip(self.genes, other.genes))
    

### Chromosome Level Constraints

```python

Objective class
1. Minimize conflict: Minimize conflicts in the schedule by ensuring that no two groups are assigned to the same lab and time slot simultaneously.
2. Maximize Resource Utilization: Maximize the utilization of assistants by distributing tasks evenly among them. Each assistant should be assigned to a balanced number of groups and shift to avoid overloading.
3. Maximize Participant Satisfaction: Maximize the satisfaction of participants by ensuring that each participant is assigned to a time slot that is available to them.

#### Objective Class for Fitness Function
##### 1. Minimize Conflicts: 
Minimize conflicts in the schedule by ensuring that no two groups are assigned to the same lab and time slot simultaneously.

Deskripsi:
- Jika dua kelompok berbeda memiliki jadwal yang sama, maka akan terjadi konflik.

In [35]:
#minimize_conflicts.py
from collections import namedtuple

class MinimizeConflicts:
    def __init__(self):
        self.conflicts = namedtuple("Conflicts", ["time_slot", "lab"])

    def calculate_conflicts(self, chromosome: Chromosome) -> int:
        conflicts_time_slot = set()
        assigned_time_slot = set()
        conflict_count = 0
        for gene in chromosome.genes:
            if (gene.laboratory, gene.time_slot) in assigned_time_slot:
                conflicts_time_slot.add(self.conflicts(gene.time_slot, gene.laboratory))
                conflict_count += 1
            else:
                assigned_time_slot.add((gene.laboratory, gene.time_slot))
        return conflict_count

    def calculate_fitness(self, chromosome: Chromosome) -> int:
        if settings.DEBUG:
            logger.info(f"Calculating conflicted group time slots for chromosome {chromosome}")
        return self.calculate_conflicts(chromosome)

##### 2. Maximize Resource Utilization:
Maximize the utilization of assistants by distributing tasks evenly among them. Each assistant should be assigned to a balanced number of groups and shift to avoid overloading.

Deskripsi:
- Menyeimbang tugas asistent dengan memastikan bahwa setiap asisten ditugaskan ke jumlah kelompok dan shift yang seimbang untuk menghindari overloading.

In [36]:
from collections import defaultdict

class MaximizeResourceUtilization:
    def __init__(self, max_groups_per_assistant: int = 2, max_shift_per_assistant: int = 2):
        self.max_groups_per_assistant = max_groups_per_assistant
        self.max_shift_per_assistant = max_shift_per_assistant
        self.assistants_assignments = defaultdict(list)
        self.assistants_shifts = defaultdict(list)
        self._overload_penalty = 0


    def calculate_assistants_assignments(self, chromosome: Chromosome) -> int:
        self.assistants_assignments.clear()
        for gene in chromosome.genes:
            self.assistants_assignments[gene.assistant].append(gene.group)
        return self.assistants_assignments
    
    def calculate_assistants_shifts(self, chromosome: Chromosome) -> int:
        self.assistants_shifts.clear()
        for gene in chromosome.genes:
            self.assistants_shifts[gene.assistant].append(gene.time_slot)
        return self.assistants_shifts
    
    def calculate_overload_penalty(self, chromosome: Chromosome) -> int:
        self._overload_penalty = 0
        self.calculate_assistants_assignments(chromosome)
        self.calculate_assistants_shifts(chromosome)
        for assistant , assignments in self.assistants_assignments.items():
            assignments_penalty = max(0, len(assignments) - self.max_groups_per_assistant)
            shift_penalty = (max(0, len(self.assistants_shifts[assistant]) - self.max_shift_per_assistant))
            self._overload_penalty += assignments_penalty + shift_penalty
        return self._overload_penalty
    
    def calculate_fitness(self, chromosome: Chromosome) -> int:
        if settings.DEBUG:
            logger.info(f"Calculating overload penalty for assistant distribution: {chromosome}")
        return self.calculate_overload_penalty(chromosome)

##### 3. Maximize Participant Satisfaction:
Maximize the satisfaction of participants by ensuring that each participant is assigned to a time slot that is available to them.

Deskripsi:
- Memaksimalkan kepuasan peserta dengan memastikan bahwa setiap peserta ditugaskan ke slot waktu yang tersedia untuk mereka. 

```python

In [37]:
from collections import defaultdict

class ParticipantAvailability:
    def __init__(self, group_data: GroupData):
        self.group_data = group_data
        self.unsatified_participants = defaultdict(list)

    def calculate_unsatisfied_participants(self, chromosome: Chromosome):
        self.unsatified_participants.clear()
        unsatified_participants_count = 0
        for gene in chromosome.genes:
            participant_availability = group_data.get_group_schedule(gene.group)
            if not participant_availability[gene.time_slot.day][gene.time_slot.shift]:
                self.unsatified_participants[gene.group].append(gene.time_slot)
                unsatified_participants_count += 1
        return unsatified_participants_count
    
    def calculate_fitness(self, chromosome: Chromosome):
        if settings.DEBUG:
            logger.info(f"Calculating unsatisfied participants schedule for chromosome {chromosome}")
        return self.calculate_unsatisfied_participants(chromosome)

# Population Initialization

In [38]:
# import fitness_function

class Population:
    def __init__(self, chromosome, fitness_function):
        self.chromosome = chromosome
        self.fitness_function = fitness_function

    def __repr__(self):
        return f"Chromosome: {self.chromosome}, Fitness Function: {self.fitness_function}"
    
    def __str__(self):
        return f"Chromosome: {self.chromosome}, Fitness Function: {self.fitness_function}"
    
    def __len__(self):
        return len(self.chromosome)
    
    def calculate_fitness(self):
        for chromosome in self.chromosome:
            chromosome.fitness = self.fitness_function.calculate_fitness(chromosome)


# Helper Functions

## Factory Method

```python

In [39]:
# Factory.py
import numpy as np
from math import ceil
from collections import namedtuple
from datetime import timedelta
# import LaboratoryData, ModuleData, GroupData, AssistantData, ChapterData
# import Constant, ConstraintChecker, Gene, Chromosome, Population

class Factory:
    def __init__(self, **kwargs):
        
        self.lab_data = kwargs.get("lab_data", LaboratoryData())
        self.module_data = kwargs.get("module_data", ModuleData())
        self.group_data = kwargs.get("group_data", GroupData())
        self.assistant_data = kwargs.get("assistant_data", AssistantData())
        self.chapter_data = kwargs.get("chapter_data", ChapterData())
        self.initialize()

    def initialize(self):
        # Caching data for faster access and less database query
        self.assistant_by_lab = defaultdict(list)
        for assistant in self.assistant_data.assistants:
            self.assistant_by_lab[assistant.laboratory.id].append(assistant.id)
            
        self.module_by_lab = defaultdict(list)
        for module in self.module_data.modules:
            self.module_by_lab[module.laboratory.id].append(module.id)

        self.group_by_module = defaultdict(list)
        for group in self.group_data.groups:
            self.group_by_module[group.module.id].append(group.id)

        self.chapter_by_module = defaultdict(list)
        for chapter in self.chapter_data.chapters:
            self.chapter_by_module[chapter.module.id].append(chapter.id)
            
        self.participant_by_group = defaultdict(list)
        for group in self.group_data.groups:
            for participant in self.group_data.participants(group.id):
                self.participant_by_group[group.id].append(participant.id)

    def create_time_slot(self, module_id):
        start = self.module_data.get_module(module_id).start_date
        end = self.module_data.get_module(module_id).end_date
        duration = (end - start).days + 1
        weeks_duration = ceil(duration / 7)
        week = np.random.randint(0, weeks_duration)
        day = np.random.choice(Constant.days)
        shift = np.random.choice(Constant.shifts)
        date = start + timedelta(days=week*7 + Constant.days.index(day))
        #date = start + timedelta(days=7 + Constant.days.index(day))
        return TimeSlot(date, day, shift)
    
    def create_time_slot_weekly(self, module_id, week):
        start = self.module_data.get_module(module_id).start_date
        day = np.random.choice(Constant.days)
        shift = np.random.choice(Constant.shifts)
        date = start + timedelta(days=week*7 + Constant.days.index(day))
        return TimeSlot(date, day, shift)

    # def generate_gene(self):
    #     while True:
    #         laboratory = np.random.choice(self.lab_data.laboratories)["id"]
    #         module = np.random.choice(self.lab_data.get_module(laboratory))["id"]
    #         module_chapter = np.random.choice(self.module_data.get_module_chapter(module))["id"]
    #         group = np.random.choice(self.module_data.get_group(module))["id"]
    #         assistant = np.random.choice(self.lab_data.get_assistant(laboratory))["id"]

    #         time_slot = self.create_time_slot(module_id=module)

    #         gene = Gene(laboratory, module, module_chapter, group, assistant, time_slot)
    #         constraint_checker = ConstraintChecker(gene)
    #         if constraint_checker.check():
    #             return gene

    def calculate_chromosome_length(self):
        chromosome_size = 0
        for module in self.module_data.modules:
            module_chapter_count = self.module_data.get_module_chapter(module.id).count()
            group_count = self.module_data.get_group(module.id).count()
            chromosome_size += module_chapter_count * group_count
        return chromosome_size
    
    def generate_chromosome(self):
        # if settings.DEBUG:
        #     logger.info("Generating chromosome...")
        genes = []
        for group in self.group_data.groups:
            for module in self.module_data.modules:
                if group.module.id == module.id:
                    # # Start of weekly chapter generation
                    # # Get weekly chapter duration
                    # start = module.start_date
                    # end = module.end_date
                    # duration = (end - start).days + 1
                    # weeks_duration = ceil(duration / 7)
                    # chapter_count = module.chapters.count()
                    # weekly_chapter_count = ceil(chapter_count / weeks_duration)
                    # # Generate weekly chapter
                    # for id in range(weekly_chapter_count):
                    #     laboratory_id = module.laboratory.id
                    #     module_id = module.id
                    #     module_chapter_id = id+1
                    #     group_id = group.id
                    #     assistant_id = np.random.choice(self.assistant_by_lab[laboratory_id])
                    #     time_slot = self.create_time_slot(module_id)
                    #     gene = Gene(laboratory_id, module_id, module_chapter_id, group_id, assistant_id, time_slot)
                    #     genes.append(gene)
                    # # End of weekly chapter generation

                    for chapter in module.chapters.all():
                        laboratory_id = module.laboratory.id
                        module_id = module.id
                        module_chapter_id = chapter.id
                        group_id = group.id
                        assistant_id = np.random.choice(self.assistant_by_lab[laboratory_id])
                        time_slot = self.create_time_slot(module_id)
                        gene = Gene(laboratory_id, module_id, module_chapter_id, group_id, assistant_id, time_slot)
                        genes.append(gene)

        chromosome = Chromosome(genes)
        return chromosome
            

    def generate_population(self, population_size):
        if settings.DEBUG:
            logger.info("Generating population...")
        chromosome = []
        for _ in range(population_size):
            if settings.DEBUG:
                logger.info(f"Generating chromosome {_+1}/{population_size}")
            chromosome.append(self.generate_chromosome())
        return Population(chromosome, None)

Test Factory

In [40]:
# #test Factory
factory = Factory()
settings.DEBUG = False
population = factory.generate_population(5)

In [41]:
# #count courses for each group
# def count_courses(population):
#     group_courses = defaultdict(list)
#     for chromosome in population.chromosome:
#         for gene in chromosome.genes:
#             group_courses[gene.group].append(gene.module_chapter)
#     return group_courses

In [42]:
# group_courses = count_courses(population)
# group_courses

In [43]:
# # test constraint checker
# # timer
# import time
# for chromosome in population.chromosome:
#     for gene in chromosome.genes:
#         constraint_checker = ConstraintChecker(gene)
#         start = time.time()
#         if not constraint_checker.check():
#             print(gene)
#             print(constraint_checker.check())
#         end = time.time()
#         print(f"Time taken for constraint checking: {end-start}")


## Fitness Function

```python

In [44]:
class FitnessFunction:
    def __init__(self, all: bool = True):
        self.minimize_conflicts = None
        self.maximize_resource_utilization = None
        self.participant_availability = None
        self.all = all

        # MaximizeResourceUtilization Parameters
        self.max_groups_per_assistant = 2
        self.max_shift_per_assistant = 16

    def initialize(self):
        self.minimize_conflicts = MinimizeConflicts()
        self.maximize_resource_utilization = MaximizeResourceUtilization(self.max_groups_per_assistant, self.max_shift_per_assistant)
        self.participant_availability = ParticipantAvailability(GroupData())
    
    def calculate_all_fitness(self, chromosome: Chromosome):
        fitness = []
        fitness.append(self.minimize_conflicts.calculate_fitness(chromosome))
        fitness.append(self.maximize_resource_utilization.calculate_fitness(chromosome))
        fitness.append(self.participant_availability.calculate_fitness(chromosome))
        return sum(fitness)

    
    def calculate_fitness(self, chromosome: Chromosome):
        if settings.DEBUG:
            logger.info(f"Calculating fitness for chromosome {chromosome}")
        fitness = self.calculate_all_fitness(chromosome)
        return fitness

Test Fitness Function

In [45]:
# #test FitnessFunction
# factory = Factory()
# population = factory.generate_population(10)
# fitness_function = FitnessFunction()
# population.fitness_function = fitness_function
# fitness_function.initialize()

In [46]:
# population.calculate_fitness()

In [47]:
# for chromosome in population.chromosome:
#     print(chromosome.fitness)

## Operator

```python

### Genetic Operators

```python

#### Mutation

```python

In [48]:
from copy import deepcopy
from random import random, randint, choice
# import Factory, Chromosome, FitnessFunction

class Mutation:
    def __init__(self, mutation_rate: float = 0.1, mutation_size: int = 1, factory: Factory = Factory()):
        self.mutation_rate = mutation_rate
        self.mutation_size = mutation_size
        self.factory = factory


    def mutate(self, parent: Chromosome) -> Chromosome:
        if settings.DEBUG:
            logger.info("Mutating...")
        if random() < self.mutation_rate:
            chromosome = deepcopy(parent)
            for _ in range(self.mutation_size):
                mutation_point = self.mutation_point(parent)
                chromosome.genes[mutation_point].time_slot = self.factory.create_time_slot(chromosome.genes[mutation_point].module)
                chromosome.genes[mutation_point].assistant = choice(self.factory.assistant_by_lab[chromosome.genes[mutation_point].laboratory])
                if settings.DEBUG:
                    logger.info(f"Mutated chromosome: {chromosome}")
            return chromosome
        else:
            if settings.DEBUG:
                logger.info(f"Chromosome not mutated: {parent}")
            return parent
        
    def mutation_point(self, parent: Chromosome) -> int:
        return randint(0, len(parent.genes) - 1)

Test Mutation

In [49]:
# #test Mutation
# factory = Factory()
# population = factory.generate_population(1)
# fitness_function = FitnessFunction()
# population.fitness_function = fitness_function
# population.calculate_fitness()

In [50]:
# mutation = Mutation()
# mutation.mutate(population.chromosome[0])

#### Cross Over

```python

In [51]:
import logging
from copy import deepcopy
import numpy as np
class Crossover:
    def __init__(self, crossover_probability: float = 0.5, number_of_crossover_points: int = 1):

        self.crossover_probability = crossover_probability
        self.number_of_crossover_points = number_of_crossover_points
        

    def crossover(self, parent1: Chromosome, parent2: Chromosome) -> Chromosome:
        if settings.DEBUG:
            logger.info("Crossover...")

        if np.random.random() > self.crossover_probability:
            if settings.DEBUG:
                logger.info("No crossover")
            return parent1, parent2

        chromosome1 = deepcopy(parent1)
        chromosome2 = deepcopy(parent2)

        for _ in range(self.number_of_crossover_points):
            point = self.crossover_point(parent1)
            if settings.DEBUG:
                logger.info(f"Crossover point: {point}")
            chromosome1.genes[point], chromosome2.genes[point] = chromosome2.genes[point], chromosome1.genes[point] 
            if settings.DEBUG:
                logger.info(f"Chromosome 1: {chromosome1}")
                logger.info(f"Chromosome 2: {chromosome2}")


        return chromosome1, chromosome2
    
    def crossover_point(self, parent1: Chromosome):
        return np.random.randint(0, len(parent1.genes) - 1)
        

Test Cross Over

In [52]:
# #test Crossover
# factory = Factory()
# population = factory.generate_population(2)

In [53]:
# crossover = Crossover()
# a,b = crossover.crossover(population.chromosome[0], population.chromosome[1])

In [54]:
# import pandas as pd
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# # display population.chromosome[0].genes[0] on pandas dataframe
# data = []
# gene = population.chromosome[0].genes[158]
# data.append([gene.laboratory, gene.module, gene.module_chapter, gene.group, gene.assistant, gene.time_slot.date, gene.time_slot.day, gene.time_slot.shift])
# df = pd.DataFrame(data, columns=["Laboratory", "Module", "Module Chapter", "Group", "Assistant", "Date", "Day", "Shift"])
# df


In [55]:
# data2 = []
# gene2 = population.chromosome[1].genes[158]
# data2.append([gene2.laboratory, gene2.module, gene2.module_chapter, gene2.group, gene2.assistant, gene2.time_slot.date, gene2.time_slot.day, gene2.time_slot.shift])
# df2 = pd.DataFrame(data2, columns=["Laboratory", "Module", "Module Chapter", "Group", "Assistant", "Date", "Day", "Shift"])
# df2

In [56]:
# data3 = []
# gene3 = a.genes[158]
# data3.append([gene3.laboratory, gene3.module, gene3.module_chapter, gene3.group, gene3.assistant, gene3.time_slot.date, gene3.time_slot.day, gene3.time_slot.shift])
# df3 = pd.DataFrame(data3, columns=["Laboratory", "Module", "Module Chapter", "Group", "Assistant", "Date", "Day", "Shift"])
# df3

In [57]:
# data4 = []
# gene4 = b.genes[158]
# data4.append([gene4.laboratory, gene4.module, gene4.module_chapter, gene4.group, gene4.assistant, gene4.time_slot.date, gene4.time_slot.day, gene4.time_slot.shift])
# df4 = pd.DataFrame(data4, columns=["Laboratory", "Module", "Module Chapter", "Group", "Assistant", "Date", "Day", "Shift"])
# df4

#### Repair

```python

In [58]:
import time
class Repair:
    def __init__(self, factory: Factory = Factory()):
        self.factory = factory
        
    def repair(self, parent: Chromosome, max_iteration=100) -> Chromosome:
        if settings.DEBUG:
            logger.info("Repairing...")
            # count = 0
            # start = time.time()
        chromosome = deepcopy(parent)
        for index, gene in enumerate(chromosome.genes):
            constraint_checker = ConstraintChecker(gene)
            if not constraint_checker.schedule_constraint():
                # if settings.DEBUG:
                #     logger.info(f"Gene {index} is not feasible, repairing...")
                feasible_time_slot = self.find_feasible_solution(gene, max_iteration)
                gene.time_slot = feasible_time_slot
                # if settings.DEBUG:
                #     logger.info(f"Feasible time slot: {gene.time_slot}")
                #     count += 1
        # if settings.DEBUG:
        #     end = time.time()
        #     logger.info(f"Repaired {count} genes in {end-start} seconds")
        return chromosome
    
    def find_feasible_solution(self, gene, max_iteration=100):
        # if settings.DEBUG:
        #     logger.info("Finding feasible solution...")

        new_gene = deepcopy(gene)
        for _ in range(max_iteration):
            feasible_time_slot = self.factory.create_time_slot(gene.module)
            new_gene.time_slot = feasible_time_slot
            constraint_checker = ConstraintChecker(new_gene)
            if constraint_checker.schedule_constraint():
                # if settings.DEBUG:
                #     logger.info("Found!")
                return feasible_time_slot
            
        if settings.DEBUG:
            logger.info("No feasible solution found, returning random time slot")
        return self.factory.create_time_slot(gene.module)

Test Repair

In [59]:
# #test Repair
# factory = Factory()
# population = factory.generate_population(1)
# repair = Repair()
# z = repair.repair(population.chromosome[0])

In [60]:
# counter = 0
# for gene in z.genes:
#     constraint_checker = ConstraintChecker(gene)
#     if not constraint_checker.schedule_constraint():
#         print("not feasible")
#         counter += 1
# print(counter)

#### Selection

```python

In [61]:
import numpy as np
from copy import deepcopy
from collections import defaultdict

class Selection:
    '''
    Description:
    Selection class for selecting chromosomes from population
    Selection methods:
    1. Tournament selection
    2. Elitism selection
    3. Roulette selection
    '''
    def __init__(self, tournament_size: int = 3, elitism_size: int = 1, elitism_offset: int = 0, elitism_range: int = 1, elitism_probability: float = 0.5, method: str = "tournament"):
        self.tournament_size = tournament_size
        self.elitism_size = elitism_size
        self.elitism_offset = elitism_offset
        self.elitism_range = elitism_range
        self.elitism_probability = elitism_probability
        self.method = method
        
    def select(self, population: Population, method: str = None, **kwargs) -> Chromosome:
        method = method or self.method
        if settings.DEBUG:
            logger.info("Selecting...")
        if method == "tournament":
            return self.tournament_selection(population)
        elif method == "elitism":
            return self.elitism_selection(population)
        elif method == "roulette":
            return self.roulette_selection(population)
        else:
            raise ValueError("Invalid selection method")
    
    def elitism_selection(self, population: Population) -> Chromosome:
        if settings.DEBUG:
            logger.info("Elitism selection...")
        sorted_population = sorted(population.chromosome, key=lambda x: x.fitness)
        return sorted_population[self.elitism_offset:self.elitism_offset + self.elitism_range]
    
    def tournament_selection(self, population: Population) -> Chromosome:
        if settings.DEBUG:
            logger.info("Tournament selection...")
        selected_chromosome = []
        for _ in range(len(population.chromosome)):
            selected_chromosome.append(self.tournament_selection_helper(population))
        return selected_chromosome[0]
    
    def tournament_selection_helper(self, population: Population) -> Chromosome:
        tournament_population = np.random.choice(population.chromosome, self.tournament_size)
        sorted_population = sorted(tournament_population, key=lambda x: x.fitness)
        return sorted_population[0]
    
    def roulette_selection(self, population: Population) -> Chromosome:
        if settings.DEBUG:
            logger.info("Roulette selection...")
        fitness = [chromosome.fitness for chromosome in population.chromosome]
        total_fitness = sum(fitness)
        probabilities = [chromosome_fitness / total_fitness for chromosome_fitness in fitness]
        selected_chromosome = np.random.choice(population.chromosome, p=probabilities)
        return selected_chromosome
    
        
    


In [62]:
# # test Selection,
# factory = Factory()
# population = factory.generate_population(10)
# fitness_function = FitnessFunction()
# population.fitness_function = fitness_function
# population.calculate_fitness()

In [63]:
# for chromosome in population.chromosome:
#     print(chromosome.fitness)

Test Selection

In [64]:
# selection = Selection(method="elitism", elitism_offset=0, elitism_range=5, elitism_probability=0.5)
# a = selection.select(population)

In [65]:
# for chromosome in a:
#     print(chromosome.fitness)

In [66]:
#test Selection
# factory = Factory()
# population = factory.generate_population(2)
# population.chromosome[0].genes
# population.chromosome[1].genes
# selection = Selection(population, method="roulette")
# a = selection.select()
    

### Genetic Algorithm

```python

In [67]:
from math import log


class GeneticAlgorithm:
    '''
    Description:
    Main class for genetic algorithm implementation
    '''
    def __init__(self, builder):
        # Dependency injection
        self.builder = builder
        self.population_size, self.max_generation = self.builder.get_population_parameters()
        self.mutation_rate, self.mutation_size = self.builder.get_mutations_parameters()
        self.crossover_probability, self.number_of_crossover_points = self.builder.get_crossover_parameters()
        self.fitness_function = self.builder.fitness_function
        self.factory = self.builder.factory
        self.mutation = self.builder.mutation
        self.crossover = self.builder.crossover
        self.selection = self.builder.selection
        self.repair = self.builder.repair
        self.logging = self.builder.logging
        # Results
        self._results = defaultdict(list)
        self._best_chromosome = None
        self._best_fitness = None
        self._best_generation = None
        self._best_chromosome_history = []
        self._best_fitness_history = []

    def show_config(self):
        if settings.DEBUG:
            logger.info("Showing configuration...")
        print(f"Population size: {self.population_size}")
        print(f"Max generation: {self.max_generation}")
        print(f"Mutation rate: {self.mutation_rate}")
        print(f"Mutation size: {self.mutation_size}")
        print(f"Crossover probability: {self.crossover_probability}")
        print(f"Number of crossover points: {self.number_of_crossover_points}")
        print(f"Fitness function: {self.fitness_function}")
        print(f"Logging: {self.logging}")
        return self

    def initialize_population(self):
        self.show_config()
        if settings.DEBUG:
            logger.info("Initializing population...")
        population = self.factory.generate_population(self.population_size)
        population.fitness_function = self.fitness_function
        population.calculate_fitness()
        return population
    
    def loggings(self, population):
        if settings.DEBUG:
            logger.info("Logging...")
        self._results["best_chromosome"].append(population.chromosome[0])
        self._results["best_fitness"].append(population.chromosome[0].fitness)
        self._results["average_fitness"].append(sum([chromosome.fitness for chromosome in population.chromosome]) / len(population.chromosome))
        self._results["worst_fitness"].append(population.chromosome[-1].fitness)
        self._best_chromosome_history.append(population.chromosome[0])
        self._best_fitness_history.append(population.chromosome[0].fitness)
        return self
    
    def evolve(self, population):
        if settings.DEBUG:
            logger.info("Evolving...")
        new_population = Population([], None)

        # Choose the best chromosome from the previous generation
        new_population.chromosome.extend(self.selection.select(population, method="elitism"))
        while len(new_population.chromosome) < self.population_size:
            parent1 = self.selection.select(population)
            parent2 = self.selection.select(population)
            child1, child2 = self.crossover.crossover(parent1, parent2)
            child1 = self.mutation.mutate(child1)
            child2 = self.mutation.mutate(child2)
            child2 = self.repair.repair(child2)
            child1 = self.repair.repair(child1)
            new_population.chromosome.extend([child1, child2])
        new_population.fitness_function = self.fitness_function
        new_population.calculate_fitness()
        if self.logging:
            self.loggings(new_population)

        return new_population
        

    def run(self):
        #if settings.DEBUG:
        logger.info("Running...")
        population = self.initialize_population()
        if self.logging:
            self.loggings(population)
        for generation in range(self.max_generation):
            #if settings.DEBUG:
            logger.info(f"Generation {generation}")
            start = time.time()
            population = self.evolve(population)
            logger.info(f"Population length: {len(population)}") 
            end = time.time()
            logger.info(f"Time taken for generation {generation}: {end-start}")
            if self.logging:
                self.loggings(population)
                logger.info(f"Best fitness: {self._results['best_fitness'][-1]}")
        self._best_chromosome = self._results["best_chromosome"][-1]
        self._best_fitness = self._results["best_fitness"][-1]
        self._best_generation = self._best_fitness_history.index(self._best_fitness)
        return self
    
    def get_results(self):
        return self._results
    
    def get_best_chromosome(self):
        return self._best_chromosome
    
    def get_best_fitness(self):
        return self._best_fitness
    
    def get_best_generation(self):
        return self._best_generation
    
    def get_best_chromosome_history(self):
        return self._best_chromosome_history
    
    def get_best_fitness_history(self):
        return self._best_fitness_history

#### Builder

In [68]:
from collections import defaultdict, namedtuple

class GeneticAlgorithmBuilder:
    '''
    Description:
    Genetic Algorithm Builder class for managing genetic algorithm parameters.
    '''
    def __init__(self):
        # Population parameters
        self.population_size = None
        self.max_generation = 10
        # Mutation parameters
        self.mutation_rate = 0.5
        self.mutation_size = 1
        # Crossover parameters
        self.crossover_probability = 0.5
        self.number_of_crossover_points = 1
        self.crossover = Crossover()
        # Selection parameters
        self.tournament_size = 3
        self.elitism_size = 1
        self.elitism_offset = 0
        self.elitism_range = 1
        self.elitism_probability = 0.5
        self.selection_method = "tournament"
        # Dependency injection
        # Fitness function
        self.fitness_function = FitnessFunction()
        # Factory
        self.factory = Factory()
        # Mutation
        self.mutation = Mutation()
        # Selection
        self.selection = Selection()
        # Repair
        self.repair = Repair()
        # Logging
        self.logging = False
        # Data
        self._lab_data = LaboratoryData()
        self._module_data = ModuleData()
        self._group_data = GroupData()
        self._assistant_data = AssistantData()
        self._chapter_data = ChapterData()

    
    
    def set_mutations_class(self, mutation):
        self.mutation = mutation
        return self

    def set_mutations(self, mutation_rate, mutation_size):
        """
        Set mutation parameters
        :param mutation_rate: Mutation rate, probability of mutation
        :type mutation_rate: float
        :param mutation_size: Number of genes to mutate
        :type mutation_size: int
        :return: self
        """
        self.mutation_rate = mutation_rate
        self.mutation_size = mutation_size
        return self
    
    def get_mutations_parameters(self):
        return self.mutation_rate, self.mutation_size
    
    def set_crossover_class(self, crossover):
        self.crossover = crossover
        return self
    
    def set_crossover(self, crossover_probability, number_of_crossover_points):
        """
        Set crossover parameters
        :param crossover_probability: Probability of crossover
        :type crossover_probability: float
        :param number_of_crossover_points: Number of points to crossover
        :type number_of_crossover_points: int
        :return: self"""

        self.crossover_probability = crossover_probability
        self.number_of_crossover_points = number_of_crossover_points
        return self
    
    def get_crossover_parameters(self):
        return self.crossover_probability, self.number_of_crossover_points
    
    def set_selection_class(self, selection):
        self.selection = selection
        return self
    
    def set_selection(self, selection_method, **kwargs):

        """
        Set selection parameters
        :param selection_method: Selection method, "tournament", "elitism" or "roulette"
        :type selection_method: str
        :param kwargs: Selection parameters, tournament_size, elitism_size, elitism_offset, elitism_range, elitism_probability
        :type kwargs: dict
        :param tournament_size: Number of chromosomes to select for tournament selection
        :type tournament_size: int
        :param elitism_size: Number of chromosomes to select for elitism selection
        :type elitism_size: int
        :param elitism_offset: Offset for elitism selection, position of the first chromosome to select
        :type elitism_offset: int
        :param elitism_range: Range for elitism selection, number of chromosomes to select from the offset
        :type elitism_range: int
        :param elitism_probability: Probability of selecting from the elitism selection
        :type elitism_probability: float
        :return: self
        """
        

        self.selection_method = selection_method
        if selection_method == "tournament":
            self.tournament_size = kwargs.get("tournament_size", 3)
        elif selection_method == "elitism":
            self.elitism_size = kwargs.get("elitism_size", 1)
            self.elitism_offset = kwargs.get("elitism_offset", 0)
            self.elitism_range = kwargs.get("elitism_range", 1)
            self.elitism_probability = kwargs.get("elitism_probability", 0.5)
        elif selection_method == "roulette":
            pass
        else:
            raise ValueError("Invalid selection method.")
        return self
    
    def get_selection_parameters(self):
        return self.tournament_size, self.elitism_size, self.elitism_offset, self.elitism_range, self.elitism_probability
    
    def set_population_parameters(self, population_size, max_generation):
        """
        Set population parameters
        :param population_size: Population size
        :type population_size: int
        :param max_generation: Maximum number of generation
        :type max_generation: int
        :return: self
        """
        self.population_size = population_size
        self.max_generation = max_generation
        return self
    
    def get_population_parameters(self):
        return self.population_size, self.max_generation
    
    def set_lab_data(self, lab_data: LaboratoryData):
        self._lab_data = lab_data
        return self
    
    def set_module_data(self, module_data: ModuleData):
        self._module_data = module_data
        return self
    
    def set_group_data(self, group_data: GroupData):
        self._group_data = group_data
        return self
    
    def set_assistant_data(self, assistant_data: AssistantData):
        self._assistant_data = assistant_data
        return self
    
    def set_chapter_data(self, chapter_data: ChapterData):
        self._chapter_data = chapter_data
        return self
    
    def initialize(self):
        self.mutation = Mutation(mutation_rate=self.mutation_rate, mutation_size=self.mutation_size, factory=self.factory)
        self.crossover = Crossover(crossover_probability=self.crossover_probability, number_of_crossover_points=self.number_of_crossover_points)
        self.selection = Selection(tournament_size=self.tournament_size, elitism_size=self.elitism_size, elitism_offset=self.elitism_offset, elitism_range=self.elitism_range, elitism_probability=self.elitism_probability, method = self.selection_method)
        self.repair = Repair(factory=self.factory)
        self.fitness_function.initialize()
        return self
    
    def build(self):
        """
        Build genetic algorithm
        :return: GeneticAlgorithm
        """
        self.initialize()
        return GeneticAlgorithm(self)


Test Genetic Algorithm

In [69]:
#test GeneticAlgorithm
builder = GeneticAlgorithmBuilder()
builder.set_population_parameters(population_size=25,
                                  max_generation=500)
builder.set_mutations(mutation_rate=0.7,
                      mutation_size=1)
builder.set_crossover(crossover_probability=0.7,
                      number_of_crossover_points=1)
builder.set_selection(selection_method="tournament",
                        tournament_size=3)

builder.fitness_function = FitnessFunction()
builder.fitness_function.max_groups_per_assistant = 15
builder.fitness_function.max_shift_per_assistant = 15
builder.logging = False
algorithm = builder.build()


In [70]:
import cProfile
settings.DEBUG = False
cp = cProfile.Profile()
cp.enable()
algorithm.run()
cp.disable()

INFO:__main__:Running...


Population size: 25
Max generation: 500
Mutation rate: 0.7
Mutation size: 1
Crossover probability: 0.7
Number of crossover points: 1
Fitness function: <__main__.FitnessFunction object at 0x000001AC3734D660>
Logging: False


INFO:__main__:Generation 0
INFO:__main__:Population length: 25
INFO:__main__:Time taken for generation 0: 190.9403555393219
INFO:__main__:Generation 1
INFO:__main__:Population length: 25
INFO:__main__:Time taken for generation 1: 73.38453602790833
INFO:__main__:Generation 2
INFO:__main__:Population length: 25
INFO:__main__:Time taken for generation 2: 72.87698554992676
INFO:__main__:Generation 3
INFO:__main__:Population length: 25
INFO:__main__:Time taken for generation 3: 74.07512044906616
INFO:__main__:Generation 4
INFO:__main__:Population length: 25
INFO:__main__:Time taken for generation 4: 72.33203458786011
INFO:__main__:Generation 5
INFO:__main__:Population length: 25
INFO:__main__:Time taken for generation 5: 72.0943853855133
INFO:__main__:Generation 6
INFO:__main__:Population length: 25
INFO:__main__:Time taken for generation 6: 73.28549671173096
INFO:__main__:Generation 7
INFO:__main__:Population length: 25
INFO:__main__:Time taken for generation 7: 72.07914328575134
INFO:__ma

KeyboardInterrupt: 

In [73]:
cp.print_stats(sort='cumtime')

         3524473201 function calls (3412798289 primitive calls) in 4750.639 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000 4747.835 

2373.917 interactiveshell.py:3472(run_code)
        2    0.000    0.000 4746.706 2373.353 {built-in method builtins.exec}
        1    0.000    0.000 4732.251 4732.251 1175231824.py:1(<module>)
        1    0.141    0.141 4732.251 4732.251 4031027849.py:87(run)
       58    0.255    0.004 4671.012   80.535 4031027849.py:63(evolve)
   840498   28.149    0.000 4545.903    0.005 2213152367.py:117(get_group_schedule)
   840498    5.965    0.000 4488.638    0.005 2213152367.py:110(get_participant_schedule)
   840498   12.699    0.000 4482.395    0.005 2213152367.py:101(participants)
  4230320   27.878    0.000 3522.293    0.001 query.py:1879(_fetch_all)
 10122909   68.250    0.000 3490.748    0.000 query.py:85(__iter__)
  4229395   34.440    0.000 2761.917    0.001 compiler.py:1532(execute_sql)
  2506417   18.874    0.000 2578.694    0.001 related_descriptors.py:201(__get__)
  2498317   14.322    0.000 2534.549    0.001 related_descriptors.py:196(get_object)
  2498317   18.887    0.000 2380

In [47]:
best_chromosome = algorithm.get_best_chromosome()

In [48]:
best_chromosome.fitness

38

In [49]:
best_chromosome.genes

[Gene(lab=1, module=1, module_chapter=1, group=1, assistant=2, time_slot=TimeSlot(date=datetime.datetime(2022, 9, 22, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Thursday', shift='Shift5')),
 Gene(lab=1, module=1, module_chapter=2, group=1, assistant=6, time_slot=TimeSlot(date=datetime.datetime(2022, 9, 21, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Wednesday', shift='Shift6')),
 Gene(lab=1, module=1, module_chapter=1, group=2, assistant=1, time_slot=TimeSlot(date=datetime.datetime(2022, 9, 23, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Friday', shift='Shift4')),
 Gene(lab=1, module=1, module_chapter=2, group=2, assistant=3, time_slot=TimeSlot(date=datetime.datetime(2022, 9, 20, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Tuesday', shift='Shift1')),
 Gene(lab=1, module=1, module_chapter=1, group=3, assistant=3, time_slot=TimeSlot(date=datetime.datetime(2022, 9, 21, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Wednesday', shift='Shift6'

##### Test Data

In [66]:
# display best chromosome on pandas dataframe, group each gene by date and shift, which the assistant as the column and show the group as the value
import pandas as pd
data = []
for gene in best_chromosome.genes:
    data.append([gene.time_slot.date, gene.time_slot.shift, gene.assistant, gene.group])
df = pd.DataFrame(data, columns=["Date", "Shift", "Assistant", "Group"])
df = df.groupby(["Date", "Shift", "Assistant"])["Group"].apply(list).reset_index()
df

NameError: name 'best_chromosome' is not defined

In [51]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
start_date = df["Date"].min()
date_interval = pd.DateOffset(days=7)

weekly_schedule = []
while start_date <= df["Date"].max():
    end_date = start_date + date_interval
    weekly_df = df[(df["Date"] >= start_date) & (df["Date"] < end_date)]
    # fill missing values with empty list
    weekly_schedule.append(weekly_df.pivot_table(index=["Date", "Shift"], columns="Assistant", values="Group", aggfunc=lambda x: x))
    start_date = end_date

for i, weekly_df in enumerate(weekly_schedule):
    print(f"Week {i+1}")
    # Format date to dd-mm-yyyy
    weekly_df.index = weekly_df.index.set_levels(weekly_df.index.levels[0].strftime('%d-%m-%Y'), level=0)
    print(weekly_df)
    print("\n")

Week 1
Assistant                1         2             3         4        5        6
Date       Shift                                                              
19-09-2022 Shift1      NaN       NaN           NaN       NaN     [33]      NaN
           Shift2      NaN       NaN           NaN       NaN      NaN  [8, 25]
           Shift3      NaN  [31, 34]           NaN      [24]      NaN      NaN
           Shift4      NaN       [4]           NaN       NaN      NaN      NaN
           Shift5      NaN       NaN          [26]       NaN     [19]      NaN
20-09-2022 Shift1      NaN       NaN           [2]       NaN      NaN      NaN
           Shift2      NaN       NaN      [20, 21]  [12, 21]     [36]      NaN
           Shift3      NaN       NaN          [14]       NaN     [24]      NaN
           Shift4     [16]       NaN           NaN       NaN      NaN      NaN
           Shift5     [31]  [13, 27]           NaN    [7, 9]      NaN     [10]
           Shift6      NaN       NaN         

In [52]:
# Count total of shifts and group for each assistant, each shift of the same day with different group is counted as 1

assistant_shifts = defaultdict(set)
assistant_groups = defaultdict(set)
assistant_modules = defaultdict(set)

for gene in best_chromosome.genes:
    assistant_shifts[gene.assistant].add((gene.time_slot.date, gene.time_slot.shift, gene.module))
    assistant_groups[gene.assistant].add((gene.time_slot.date, gene.time_slot.shift, gene.group, gene.module))


#sort by assistant id
assistant_shifts = dict(sorted(assistant_shifts.items()))
for assistant in assistant_shifts:
    assistant_name = assistant_data.get_assistant(assistant)
    print(f"Assistant {assistant_name} of {assistant_name.laboratory}")
    print(f"Total shifts: {len(assistant_shifts[assistant])}")
    print(f"Total groups: {len(assistant_groups[assistant])}")
    print("---------------------------------")

Assistant Fatimah of Dasar Elektro
Total shifts: 10
Total groups: 11
---------------------------------
Assistant Rahmat of Dasar Elektro
Total shifts: 11
Total groups: 14
---------------------------------
Assistant Amel of Dasar Elektro
Total shifts: 10
Total groups: 15
---------------------------------
Assistant Bowo of Dasar Elektro
Total shifts: 9
Total groups: 11
---------------------------------
Assistant Rafli of Dasar Elektro
Total shifts: 11
Total groups: 12
---------------------------------
Assistant Andi of Dasar Elektro
Total shifts: 8
Total groups: 9
---------------------------------


# Tabu Search

```python

In [53]:
#tabu search
import collections
from copy import deepcopy
import numpy as np
import itertools
import random

## Tabu List

In [54]:
# Tabu Search Operators
import collections
class TabuList:
    def __init__(self, tabu_list_size):
        """"
        Tabu list class for managing tabu list
        :param tabu_list_size: Tabu list size
        :type tabu_list_size: int
        """
        self.tabu_list_size = tabu_list_size
        self.tabu_list = collections.deque(maxlen=tabu_list_size)
    
    def __repr__(self):
        return f"Tabu List Size: {self.tabu_list_size}, Tabu List: {self.tabu_list}"
    
    def __str__(self):
        return f"Tabu List Size: {self.tabu_list_size}, Tabu List: {self.tabu_list}"
    
    def is_tabu(self, chromosome: Chromosome):
        '''
        Check if chromosome is in tabu list
        :param chromosome: Chromosome
        :type chromosome: Chromosome
        :return: bool
        '''
        return chromosome in self.tabu_list
        
    
    def clear(self):
        '''
        Clear tabu list
        :return: None
        '''
        self.tabu_list.clear()
    
    def update(self, chromosome):
        '''
        Update tabu list
        :param chromosome: Chromosome
        :type chromosome: Chromosome
        :return: None
        '''
        self.tabu_list.append(chromosome)

## Neighborhood Operation

In [72]:
# Neighborhood Operators

import itertools
from copy import deepcopy
import numpy as np
class Neighborhood:
    def __init__(self, tabu_list: TabuList):
        self.tabu_list = tabu_list
    
    def swap(self, chromosome: Chromosome):
        '''
        Swap two adjacent genes
        :param chromosome: Chromosome
        :type chromosome: Chromosome
        :return: Chromosome
        '''
        if settings.DEBUG:
            logger.info("Swapping...")
        new_chromosome = deepcopy(chromosome)
        max_distance = min(2, len(new_chromosome.genes) - 1)
        swap_attempt = min(2, len(new_chromosome.genes) // 2)

        for _ in range(swap_attempt):
            gene1, gene2 = np.random.choice(new_chromosome.genes, 2, replace=False)
            if abs(new_chromosome.genes.index(gene1) - new_chromosome.genes.index(gene2)) <= max_distance:
                if gene1.module == gene2.module:
                    gene1.assistant, gene2.assistant = gene2.assistant, gene1.assistant
                    gene1.time_slot, gene2.time_slot = gene2.time_slot, gene1.time_slot
                if not self.tabu_list.is_tabu(new_chromosome):
                    logger.info("Swapped! gene1 index: {}, gene2 index: {}".format(new_chromosome.genes.index(gene1), new_chromosome.genes.index(gene2)))
                    return new_chromosome
                else:
                    if gene1.module == gene2.module:
                        gene1.assistant, gene2.assistant = gene2.assistant, gene1.assistant
                        gene1.time_slot, gene2.time_slot = gene2.time_slot, gene1.time_slot
    
    def swap_random(self, chromosome: Chromosome):
        '''
        Swap two random genes
        :param chromosome: Chromosome
        :type chromosome: Chromosome
        :return: Chromosome
        '''
        if settings.DEBUG:
            logger.info("Swapping random...")
        new_chromosome = deepcopy(chromosome)
        swap_attempt = min(2, len(new_chromosome.genes) // 2)
        for _ in range(swap_attempt):
            gene1, gene2 = np.random.choice(new_chromosome.genes, 2, replace=False)
            if gene1.module == gene2.module:
                gene1.assistant, gene2.assistant = gene2.assistant, gene1.assistant
                gene1.time_slot, gene2.time_slot = gene2.time_slot, gene1.time_slot
            if not self.tabu_list.is_tabu(new_chromosome):
                logger.info("Swapped! gene1 index: {}, gene2 index: {}".format(new_chromosome.genes.index(gene1), new_chromosome.genes.index(gene2)))
                return new_chromosome
            else:
                if gene1.module == gene2.module:
                    gene1.assistant, gene2.assistant = gene2.assistant, gene1.assistant
                    gene1.time_slot, gene2.time_slot = gene2.time_slot, gene1.time_slot

        
    def run(self, chromosome: Chromosome):
        '''
        Run neighborhood operators
        :param chromosome: Chromosome
        :type chromosome: Chromosome
        :return: list
        '''
        if settings.DEBUG:
            logger.info("Running neighborhood operators...")
        neighborhood = []
        neighborhood.append(self.swap(chromosome))
        #neighborhood.append(self.swap_random(chromosome))

        neighborhood = [neighbor for neighbor in neighborhood if neighbor is not None]
        return neighborhood
        
    


In [112]:
# Test Neighborhood
tabu_list = TabuList(tabu_list_size=10)
neighborhood = Neighborhood(tabu_list)
best_chromosome = algorithm.get_best_chromosome()

len(best_chromosome.genes)

72

## Search Strategy

```python

In [68]:
# Strategy Operators
# Best Improvement, First Improvement, Random

class Strategy:
    def __init__(self, tabu_list: TabuList, fitness_function: FitnessFunction):
        self.tabu_list = tabu_list
        self.fitness_function = fitness_function
    
    def best_improvement(self, neighborhood: list) -> Chromosome:
        '''
        Best improvement strategy
        :param neighborhood: Neighborhood
        :type neighborhood: list
        :return: Chromosome
        '''
        if settings.DEBUG:
            logger.info("Best improvement...")
        best_chromosome = None
        best_fitness = 0
        for chromosome in neighborhood:
            fitness = self.fitness_function.calculate_fitness(chromosome)
            logger.info(f"Fitness: {fitness}, Best fitness: {best_fitness}")

            if fitness > best_fitness:
                best_fitness = fitness
                best_chromosome = chromosome
        return best_chromosome
    
    def first_improvement(self, neighborhood: list) -> Chromosome:
        '''
        First improvement strategy
        :param neighborhood: Neighborhood
        :type neighborhood: list
        :return: Chromosome
        '''
        if settings.DEBUG:
            logger.info("First improvement...")
        for chromosome in neighborhood:
            if not self.tabu_list.is_tabu(chromosome):
                return chromosome
        return None
    
    def random(self, neighborhood: list) -> Chromosome:
        '''
        Random strategy
        :param neighborhood: Neighborhood
        :type neighborhood: list
        :return: Chromosome
        '''
        if settings.DEBUG:
            logger.info("Random...")
        return random.choice(neighborhood)
    
    def run(self, neighborhood: list, method: str = "best_improvement") -> Chromosome:
        '''
        Run strategy
        :param neighborhood: Neighborhood
        :type neighborhood: list
        :return: Chromosome
        '''
        if method == "best_improvement":
            selected_strategy = self.best_improvement
        elif method == "first_improvement":
            selected_strategy = self.first_improvement
        elif method == "random":
            selected_strategy = self.random
        return selected_strategy(neighborhood)

## Tabu Search Algorithm

```python

In [69]:
# Tabu Search
class TabuSearch:
    def __init__(self, tabu_list: TabuList, neighborhood: Neighborhood, strategy: Strategy, fitness_function: FitnessFunction, max_iteration: int = 100):
        self.tabu_list = tabu_list
        self.neighborhood = neighborhood
        self.strategy = strategy
        self.max_iteration = max_iteration
        self.fitness_function = fitness_function
        self.fitness_function.initialize()
        
    
    def search(self, chromosome: Chromosome, method: str = "swap") -> Chromosome:
        '''
        Tabu search
        :param chromosome: Chromosome
        :type chromosome: Chromosome
        :return: Chromosome
        '''
        self.tabu_list.clear()
        if settings.DEBUG:
            logger.info("Tabu search...")
        iteration = 0
        best_chromosome = chromosome
        while iteration < self.max_iteration:
            iteration += 1
            neighborhood = self.neighborhood.run(best_chromosome)
            logger.info(f"Neighborhood size: {len(neighborhood)}")
            best_chromosome = self.strategy.run(neighborhood, method="best_improvement")
            self.tabu_list.update(best_chromosome)
            logger.info(f"Iteration {iteration}, Best fitness: {best_chromosome.fitness}")
        return best_chromosome

In [75]:
chromosome = best_chromosome
tabu_list = TabuList(tabu_list_size=5)
fitness_function = FitnessFunction()
fitness_function.max_groups_per_assistant = 15
fitness_function.max_shift_per_assistant = 15
neighborhood = Neighborhood(tabu_list)
strategy = Strategy(tabu_list, fitness_function)
tabu_search = TabuSearch(tabu_list, neighborhood, strategy, max_iteration=20, fitness_function=fitness_function)
settings.DEBUG = False
best_chromosome_tabu = tabu_search.search(chromosome, method="swap")

INFO:__main__:Neighborhood size: 0


AttributeError: 'NoneType' object has no attribute 'fitness'

False

In [ ]:
class TabuSearchBuilder:
    def __init__(self):
        self.tabu_list_size = None
        self.max_iteration = None
        self.neighborhood_method = None
        self.search_strategy = None
        self.factory = Factory()
        self.fitness_function = FitnessFunction()
        self.tabu_list = TabuList(self.tabu_list_size)
        self.neighborhood = Neighborhood()
        self.logging = False
        self._results = defaultdict(list)
        self._best_chromosome = None
        self._best_fitness = None
        self._best_iteration = None
        self._best_chromosome_history = []
        self._best_fitness_history = []
    
    def set_tabu_list_size(self, tabu_list_size):
        self.tabu_list_size = tabu_list_size
        return self
    
    def set_max_iteration(self, max_iteration):
        self.max_iteration = max_iteration
        return self
    
    def set_neighborhood_method(self, neighborhood_method):
        self.neighborhood_method = neighborhood_method
        return self
    
    def set_search_strategy(self, search_strategy):
        self.search_strategy = search_strategy
        return self
    
    def set_factory(self, factory):
        self.factory = factory
        return self
    
    def set_fitness_function(self, fitness_function):
        self.fitness_function = fitness_function
        return self
    
    def set_logging(self, logging):
        self.logging = logging
        return self
    
    def get_results(self):
        return self._results
    
    def get_best_chromosome(self):
        return self._best_chromosome
    
    def get_best_fitness(self):
        return self._best_fitness
    
    def get_best_iteration(self):
        return self._best_iteration
    
    def get_best_chromosome_history(self):
        return self._best_chromosome_history
    
    def get_best_fitness_history(self):
        return self._best_fitness_history
    
    def build(self):
        if self.tabu_list_size is None:
            raise ValueError("Tabu list size is not set")
        if self.max_iteration is None:
            raise ValueError("Max iteration is not set")
        if self.neighborhood_method is None:
            raise ValueError("Neighborhood method is not set")
        if self.search_strategy is None:
            raise ValueError("Search strategy is not set")
        if self.factory is None:
            raise ValueError("Factory is not set")
        if self.fitness_function is None:
            raise ValueError("Fitness function is not set")
        return TabuSearch(self)

In [ ]:
# #test TabuSearch
builder = TabuSearchBuilder()
builder.tabu_list_size = 10
builder.max_iteration = 100
builder.neighborhood_method = "swap"
builder.search_strategy = FirstImprovementStrategy(max_iteration=100)
builder.factory = Factory()
builder.fitness_function = FitnessFunction()
builder.logging = True
tabu_search = builder.build()